In [128]:
import pandas as pd
import numpy as np

In [129]:
movies = pd.read_csv('movies.csv')


In [130]:
import re

In [131]:
def clean_title(title):
    return re.sub("[^a-zA-Z0-9 ]"," ", title)
    

In [132]:
movies["clean_title"] = movies["title"].apply(clean_title)

In [133]:
movies

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
62418,209157,We (2018),Drama,We 2018
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul 2001
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems 2018
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing 2001


In [134]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(ngram_range = (1,2))

In [ ]:
# turnig the set of title into a matrix
tfidf = vectorizer.fit_transform(movies["clean_title"])

In [ ]:
tfidf

creating the search function

In [135]:
from sklearn.metrics.pairwise import cosine_similarity 
import numpy as np
def search(title):
    title = clean_title(title)
    #changing the title set to vector matrix
    query_vec = vectorizer.transform([title])
    #finding the similarity to comapare the query of term from each of the title
    similarity = cosine_similarity(query_vec,tfidf).flatten()
    # find the title with the highest similarity
    indices = np.argpartition(similarity,-5)[-5:]
    #index the title
    result = movies.iloc[indices][::1]
    return result

building an interactive search engine using Jupyter

In [136]:
import ipywidgets as widgets
from IPython.display import display
#creating the input widget
movie_input = widgets.Text(
    value = "Toy story",
    description = "movie Title",
    disabled = False
)
movie_output_list = widgets.Output()
movie_input
#call this function when anything is type
def on_type(data):
    with movie_output_list:
        movie_output_list.clear_output()
        title= data["new"]
        if len(title) > 5:
            display(search(title))
# run on_type function if there is any change in the i nput
#hooking the input function with on_type
movie_input.observe(on_type, names="value")
display(movie_input,  movie_output_list)


Text(value='Toy story', description='movie Title')

Output()

Creating the recommendation system

In [137]:
rating = pd.read_csv("ratings.csv")

MemoryError: Unable to allocate 572. MiB for an array with shape (3, 25000095) and data type int64

In [ ]:
rating.head()